# Function(ality) replacement exploration (TF generic)

## Table of content (ToC)<a class="anchor" id="TOC"></a>
* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Some methods</a>
   * <a href="#bullet2x1">2.1 - Method 1: Replacement function</a>
       * <a href="#bullet2x1x1">2.1.1 - Demo code</a>
       * <a href="#bullet2x1x2">2.1.2 - Pro/cons</a>
   * <a href="#bullet2x2">2.2 - Method 2: Direct reasignment</a>
       * <a href="#bullet2x2x1">2.2.1 - Demo code</a>
       * <a href="#bullet2x2x2">2.2.2 - Pro/cons</a>
   * <a href="#bullet2x3">2.3 - Method 3: Decorators</a>
       * <a href="#bullet2x3x1">2.3.1 - Demo code</a>
       * <a href="#bullet2x3x2">2.3.2 - Pro/cons</a>
   * <a href="#bullet2x4">2.4 - Method 4: Monkey-patching</a>
       * <a href="#bullet2x4x1">2.4.1 - Demo code</a>
       * <a href="#bullet2x4x2">2.4.2 - Pro/cons</a>
   * <a href="#bullet2x5">2.5 - Method 5: Using types.MethodType</a>
       * <a href="#bullet2x5x1">2.5.1 - Demo code</a>
       * <a href="#bullet2x5x2">2.5.2 - Pro/cons</a>
       

       
       
* <a href="#bullet6">6 - Notebook version details</a> 

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

In Python, being able to replace or tweak functions on the fly is a handy technique, which can be used for both debugging and for changing function behaviour. There are different ways to do this, each with its own strengths and limitations. This notebook is intended to examine them in order to be able to determine the most appropriate one to use in `app.py` for our multi syntax enabled Text-Farbic dataset.

# 2 - Some methods <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

There are many ways to replace a specific function or feature of Text-Fabric. Each option has its own advantages and disadvantages. In this section, we’ll look at a few possible alternatives and briefly discuss the pros and cons of each one.  

## 2.1 - Method 1: Replacement function <a class="anchor" id="bullet2x1"></a>

### 2.1.1 - Demo code <a class="anchor" id="bullet2x1x1"></a>

This method replaces an existing function with a new one while the program is running. It uses Python’s dynamic features to swap out the old function (`originalFunction1`) with a new one (`newFunction1`) by updating the reference in the function’s global scope. Some explanatory comments are added within the code, which is an actualy working code.

In [1]:
import types

def originalFunction1():
    """
    Original function that prints a message.
    """
    print("This is the original function 1.")

def newFunction1():
    """
    New function that replaces the original one.
    """
    print("This is the new function 1.")

def replaceFunction1(target, replacement):
    """
    Replace a function with another function dynamically.

    Args:
        target (function): The original function to be replaced.
        replacement (function): The new function to use as replacement.

    Returns:
        None
    """
    targetGlobals = target.__globals__
    targetName = target.__name__
    targetGlobals[targetName] = replacement

The following cell will demonstrate the function replacement:

In [2]:
if __name__ == "__main__":
    print("\nBefore replacement:")
    originalFunction1()

    print("\nPerforming function replacement...")
    replaceFunction1(originalFunction1, newFunction1)

    print("\nAfter replacement:")
    originalFunction1()


Before replacement:
This is the original function 1.

Performing function replacement...

After replacement:
This is the new function 1.


In [3]:
print("\nBefore replacement:")
originalFunction1()

print("\nPerforming function replacement...")
replaceFunction1(originalFunction1, newFunction1)

print("\nAfter replacement:")
originalFunction1()


Before replacement:
This is the new function 1.

Performing function replacement...

After replacement:
This is the new function 1.


### 2.1.2 - Pro/cons <a class="anchor" id="bullet2x1x2"></a>

One of the positive points is that this method is very simple and direct as it can change the behaviour without touching/changing the original code. Since this allows for swapping functions at runtime, it is extremely handy to test or applying a quick fix to a third-party library.

The biggest risk is that it can make the code harder to follow, especially if function replacement happens silently or (unintended) at multiple places. It could easily lead to hard to debug problems since one expects the original function still to be in place.

## 2.2 - Method 2: Direct reasignment <a class="anchor" id="bullet2x2"></a>

This method replaces the original function (`originalFunction2`) by directly assigning it to a new one (`newFunction2`). From that point on, any call to `originalFunction2()` will actually run `newFunction2()` instead.

### 2.2.1 Demo code<a class="anchor" id="bullet2x2x1"></a>

Setting up the demo is very simple. Let's first define the two functions.

In [4]:
def originalFunction2():
    print("This is the original function 2.")

def newFunction2():
    print("This is the new function 2.")

Demonstrating the function replacement:



In [5]:
# Call the function
originalFunction2()

print("\nPerforming function replacement...")
originalFunction2 = newFunction2

# Call the function again
originalFunction2()

This is the original function 2.

Performing function replacement...
This is the new function 2.


### 2.2.2 - Pro/cons <a class="anchor" id="bullet2x2x2"></a>

One of the advantages is that this approach is simple and easy to understand. It’s a straightforward aproach in most situations and requiring no external libraries, so it likely works in any Python environment. If properly documented replacing the function is clear.

There are also some downsides: as the change is global it will affecting all references to the original function, which can make debugging tricky (esp. when not properly documented). Furthermore this overwrites important metadata like __name__ and __doc__, potentially making (debug and documentation) tooling less effective.

## 2.3 - Method 3: Decorators <a class="anchor" id="bullet2x3"></a>

Decorators offer another way to replace or modify functions while your program is running. A decorator is a special kind of function that takes another function, adds something to it (or changes it), and then gives back a new version. Note: although the term decorator is unique for Python, the concept is also available in e.g. javaScript (higher-order functions).

When we apply a decorator to replace a function, it works by wrapping the original function with a new one. This wrapper function controls what happens when the original function is called. It can add new behavior, change the result, or even completely ignore the original function. However, the nice thing is that the decorator can still keep information about the original function, like its name and documentation.



### 2.3.1 - Demo code <a class="anchor" id="bullet2x3x1"></a>

In this demo code the decorator (`replaceFunction3`) returns the wrapper() function. That means the original function gets replaced with this new wrapped version. 

In [6]:
# Define the original function
def originalFunction3():
    print("This is the original function.")

# Define a decorator for function replacement
def replaceFunction3(func):
    def wrapper():
        print("This function is replaced.")
    return wrapper

Demonstrating the function replacement. In this the `@replaceFunction3` syntax passes the function `originalFunction3` as an argument to `replaceFunction3`.

In [7]:
# Call the function
originalFunction3()

print("\nPerforming function replacement...")

@replaceFunction3
def originalFunction3():
    print("This is the original function.")

# Call the function again
originalFunction3()  # Output: This function is replaced.

This is the original function.

Performing function replacement...
This function is replaced.


### 2.3.2 - pro/cons <a class="anchor" id="bullet2x3x2"></a>

As a positive, using a decorator to replace or modify a function can make the code more structured and easier to read (assuming the concept wrapper is understood). It also encourages consistent patterns and can be reused across multiple functions. With functools.wraps, we can even keep the original function’s name and docstring. 

On the downside, decorators can be a bit tricky for beginners to grasp. They also add extra code, which might feel like overkill for simple cases. And when something goes wrong, debugging can be harder because there's another layer to dig through.

## 2.4 - Method 4: Monkey-patching <a class="anchor" id="bullet2x4"></a>

Monkey-patching involves modifying or extending a function or method in a module or class at runtime, without changing the original source code.
It can be use to fix bugs, change behavior, or even add new features (either temporarily or for testing).

### 2.4.1 - Demo code <a class="anchor" id="bullet2x4x1"></a>

This demo code replaces the original `some_function` in `some_module` with the desired `new_function`. From now on, whenever `some_module.some_function()` is called, it will actually run `new_function()` instead

### 2.4.2 - pro/cons <a class="anchor" id="bullet2x4x2"></a>

The nice thing is that this method can be aplied quick and easy. It allows to customize the behavior of an existing library without changing the original source code. It's powerful for debugging, extending functionality or tweaking output. 

However, like all the previously described dynamic replacements, this method can also easily lead to confusion. Since it makes the internal flow of the code less transparent and harder to debug, this needs to be clearly documented.

## 2.5 - Method 5: Using types.MethodType <a class="anchor" id="bullet2x5"></a>

When working with classes in Python it is also possible to replace a method only on a specific instance. This can be done by using `types.MethodType`. This allows for binding a new function to that instance as if it were a normal method.

### 2.5.1 - Demo code <a class="anchor" id="bullet2x5x1"></a>

In this functioning example, a class `MyClass` has a method `my_method` that just prints "Original method." Next a new function `new_method` is defined and then attached to a single instance `obj` using `types.MethodType`. This replaces `my_method` only for that instance. Now, when `obj.my_method()` is called, it runs the new method instead. Other instances of the class are not affected and will still use the original method.

In [9]:
import types

class MyClass:
    def my_method(self):
        print("Original method.")

def new_method(self):
    print("New method.")

# Create an instance
obj = MyClass()

# Replace the method
obj.my_method = types.MethodType(new_method, obj)

# Call the new method
obj.my_method()  # Output: New method.

New method.


### 2.5.2 - pro/cons <a class="anchor" id="bullet2x5x2"></a>

The biggest pro, especialy in the context of Text-Fabric, is that method changes the behavior for just one object without affecting the whole class. it preserves `self`: types.MethodType correctly binds the method to the instance, so `self` keeps working as expected. Like all of the examples in this notebook, it also does not require to change the original code (or class in this case).

One drawback is that developers may expect that all methods on instances of a class to behave the same, whic might not be the case here (if both a tweaked TF dataset and a untweaked TF dataset is loaded). Furthermore this does require some addional imports (like types). Also this concept might not work for certain methods (possibly static or class methods?).

# Used method for TF API behaviour modification

Any of the above discussed methods do have their pro and cons that are largely similair. For the custom code that will be added to the TF datset a method replacement done at the instance level within a custom subclass of App from text-fabric. This is the essential part of the code:

from tf.advanced.app import App
from tf.advanced.display import displaySetup, displayReset
import re

class TfApp(App):

	def __init__(self, *args, **kwargs):
		super().__init__(*args, **kwargs)
		# Repalce the original show and search method and insert our new ones using the name of the original method
		import types
		self.search = types.MethodType(custom_search, self)
		self.show = types.MethodType(custom_show, self)
		self.dm('Custom versions for search and show are loaded')



In this a custom class TfApp is created which extends the default App. Inside the `__init__ method`, types.MethodType() is used to dynamically replace the instance methods `search` and `show` with custom versions (`custom_search` and `custom_show` defined in the rest of the file). These new functions are bound directly to the instance (self), effectively overriding the original methods for that object.

The main reason for using thais method is that it is NOT a global function/method replacement, but bound to an instance. That implies that it allows running multiple instances of Text-Fabric at the same time without undesired interactions between the instances.  Also I believe it to be the most transparent method that can be documented clearly in the `app.py` file. 

The indepences of the instances is demonstrated in [this notebook](test_two_TF_datasets.ipynb).

# 6 - Notebook version details<a class="anchor" id="bullet6"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.1</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>10 April 2025</td>
    </tr>
  </table>
</div>